In [2]:
import numpy as np
import pandas as pd 
import bokeh.io
import bokeh.plotting
import scikit_posthocs as posthoc

bokeh.io.output_notebook()

import urllib.request
# urllib.request.urlretrieve('https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set3/boosting_helper.py', 'boosting_helper.py')

Loading BokehJS ...

In [3]:
df_marm = pd.read_csv("raw-data/extended-data-fig6.csv")
df_marm["points"] = df_marm["copies/microgram"]

df_plot = df_marm.groupby(["tissue", "variant"])[["points"]].agg(mean=("points", "mean"),
                                                                        std=("points", "std"),
                                                                        n=("points", "count")).reset_index()
df_plot["error"] = df_plot["std"] / df_plot["n"].apply(np.sqrt)
df_plot = pd.merge(df_marm[["tissue", "variant", "points"]], df_plot, on=["tissue", "variant"])

df_plot["upper"] = df_plot.apply(lambda x: (x["mean"] + x["error"] if x["n"] > 2 else -1), axis=1) 
df_plot["lower"] = df_plot.apply(lambda x: (x["mean"] - x["error"] if x["n"] > 2 else -1), axis=1)

rng = np.random.default_rng()
df_plot['jitter'] = rng.normal(0, 0.05, len(df_plot)) 
df_plot['cats'] = df_plot.apply(lambda x: (x['variant'], x['jitter']), axis=1)

df_plot.head()

,tissue,variant,points,mean,std,n,error,upper,lower,jitter,cats
0,brain,AAV9,1.299825e+08,1.198923e+08,6.638735e+07,11,2.001654e+07,1.399088e+08,9.987576e+07,0.041586,"(AAV9, 0.04158613279832212)"
1,brain,AAV9,1.027104e+08,1.198923e+08,6.638735e+07,11,2.001654e+07,1.399088e+08,9.987576e+07,-0.041626,"(AAV9, -0.041625978637434655)"
2,brain,AAV9,1.033182e+08,1.198923e+08,6.638735e+07,11,2.001654e+07,1.399088e+08,9.987576e+07,0.078711,"(AAV9, 0.07871079956368843)"
3,brain,AAV9,1.019786e+08,1.198923e+08,6.638735e+07,11,2.001654e+07,1.399088e+08,9.987576e+07,0.029169,"(AAV9, 0.029168959431996752)"
4,brain,AAV9,1.244151e+08,1.198923e+08,6.638735e+07,11,2.001654e+07,1.399088e+08,9.987576e+07,0.045719,"(AAV9, 0.0457185692250195)"


In [10]:
x_range = ["AAV9", "CAP-Mac"]

variant_color = bokeh.transform.factor_cmap('variant', palette=["gray", "white"], factors=["AAV9", "CAP-Mac"])

markers = ['circle_x', 'plus'] 
marker_color = ["white", "black"]

############### PLOT SETTINGS ###################
figure_width = 1000
figure_height = 300
width = 1
dodge = 0

marker_size = 5
error_size=4
error_line_width=0.5
############### PLOT SETTINGS ###################

p = bokeh.plotting.figure(x_range=x_range, height=figure_height, width=figure_width, title="DNA quantification in adult marmoset brain (intravenous delivery)")

############### PLOT SETTINGS ###################
p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=45
p.axis.major_tick_in = 0
p.axis.major_label_text_color = "#000000"
p.axis.axis_label_text_align = "right"
p.add_layout(bokeh.models.Legend(), "right")
############### PLOT SETTINGS ###################


p.vbar(source=df_plot.loc[df_plot['variant']=='AAV9'], x=bokeh.transform.dodge('variant', -dodge, range=p.x_range), 
       top='mean', width=width, fill_color=variant_color, line_color='black', legend_label="AAV9")
p.vbar(source=df_plot.loc[df_plot['variant']=='CAP-Mac'], x=bokeh.transform.dodge('variant', dodge, range=p.x_range), 
       top='mean', width=width, fill_color=variant_color, line_color='black', legend_label="CAP-Mac")

source_error_AAV9 = bokeh.models.ColumnDataSource(data=df_plot.loc[df_plot['variant']=='AAV9'])
source_error_C1 = bokeh.models.ColumnDataSource(data=df_plot.loc[df_plot['variant']=='CAP-Mac'])

w_AAV9 = bokeh.models.Whisker(source=source_error_AAV9, base=bokeh.transform.dodge('variant', -dodge, range=p.x_range), 
                              upper='upper', lower='lower', level='overlay', line_width=error_line_width)
w_C1 = bokeh.models.Whisker(source=source_error_C1, base=bokeh.transform.dodge('variant', dodge, range=p.x_range), 
                            upper='upper', lower='lower', level='overlay', line_width=error_line_width)

w_AAV9.upper_head.line_width=error_line_width
w_AAV9.upper_head.size=error_size
w_AAV9.lower_head.line_width=error_line_width
w_AAV9.lower_head.size=error_size

w_C1.upper_head.line_width=error_line_width
w_C1.upper_head.size=error_size
w_C1.lower_head.line_width=error_line_width
w_C1.lower_head.size=error_size

p.add_layout(w_AAV9)
p.add_layout(w_C1)

p.scatter(x=bokeh.transform.dodge('cats', -dodge, range=p.x_range), y='points', 
          source=df_plot.loc[df_plot['variant']=='AAV9'], marker="circle", alpha=0.65,
          size=marker_size, color='black', line_color='black', line_width=0)

p.scatter(x=bokeh.transform.dodge('cats', dodge, range=p.x_range), y='points', 
          source=df_plot.loc[df_plot['variant']=='CAP-Mac'], marker="circle", alpha=0.65,
          size=marker_size, color='black', line_width=0)


p.y_range = bokeh.models.Range1d(0, np.max(np.max([df_plot["upper"], df_plot["points"]]))*1.1)
bokeh.io.show(p)